In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random as python_random

import imageio as iio
import json

from tcn import TCN

2024-11-23 07:39:18.663272: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 07:39:18.961277: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-23 07:39:18.961301: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-23 07:39:18.962649: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 07:39:19.123768: I tensorflow/core/platform/cpu_feature_g

In [2]:
tf.random.set_seed(42)
np.random.seed(42)
python_random.seed(42)

In [3]:
class CancelOut(tf.keras.layers.Layer):
    '''
    CancelOut Layer
    '''
    def __init__(self, activation='sigmoid', cancelout_loss=True, lambda_1=0.002, lambda_2=0.001):
        super(CancelOut, self).__init__()
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        self.cancelout_loss = cancelout_loss
        
        if activation == 'sigmoid':
            self.activation = tf.keras.activations.sigmoid
        elif activation == 'softmax':
            self.activation = tf.keras.activations.softmax
        else:
            raise ValueError("Unsupported activation function")

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1],),
            initializer=tf.keras.initializers.Constant(1),
            trainable=True)
        
    def call(self, inputs):
        if self.cancelout_loss:
            self.add_loss(self.lambda_1 * tf.norm(self.w, ord=1) + self.lambda_2 * tf.norm(self.w, ord=2))
        return tf.math.multiply(inputs, self.activation(self.w))
    
    def get_config(self):
        config = super(CancelOut, self).get_config()
        config.update({
            "activation": self.activation,
            "lambda_1": self.lambda_1,
            "lambda_2": self.lambda_2,
            "cancelout_loss": self.cancelout_loss
        })
        return config


In [4]:
class MyModelTCN(tf.keras.Model):

    def __init__(self, window, feature_size):
        super(MyModelTCN, self).__init__()

        # Feature Selection
        self.fs = CancelOut(activation='sigmoid', cancelout_loss=True, lambda_1=0.002, lambda_2=0.001)
        self.reshape_lstm = tf.keras.layers.Reshape((window, feature_size))

        # Part for extract topographie
        self.conv1 = tf.keras.layers.Conv2D(16, (3, 3), strides=2, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(8, (3, 3), strides=2, activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(8, (3, 3), activation='relu')
        self.pool = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense_cnn = tf.keras.layers.Dense(64, activation='relu', name='topo_dense')

        # Part for extract features
        self.tcn = TCN(64, return_sequences=True)
        self.tcn2 = TCN(64, return_sequences=False)
        self.dense = tf.keras.layers.Dense(128, activation='relu')

        # Post Merging
        self.dense_pm = tf.keras.layers.Dense(128, activation='relu')
        self.dense_pm1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense_pm2 = tf.keras.layers.Dense(32, activation='relu')
        self.dense_pm3 = tf.keras.layers.Dense(1, activation='linear')

    def call(self, x):
        '''
          x = (topographie data 2D, Features LSTM sized)
        '''
        topo, features = x
        # Extract topographie
        t = self.conv1(topo)
        t = self.pool(t)
        t = self.conv2(t)
        t = self.pool(t)
        t = self.conv3(t)
        t = self.pool(t)
        t = self.flatten(t)
        t = self.dense_cnn(t)

        # Extract features
        # f = self.flat_lstm(features)
        # f = self.fs(f)
        # f = self.reshape_lstm(f)
        f = self.flatten(features)
        f = self.fs(f)
        f = self.reshape_lstm(f)
        f = self.tcn(f)
        f = self.tcn2(f)
        f = self.dense(f)

        # Merging
        o = tf.keras.layers.concatenate([t, f])

        # Post Merging
        o = self.dense_pm(o)
        o = self.dense_pm1(o)
        o = self.dense_pm2(o)
        o = self.dense_pm3(o)

        return o

In [5]:
class MyModelTCNNotHybrid(tf.keras.Model):

    def __init__(self, window, feature_size):
        super(MyModelTCNNotHybrid, self).__init__()

        # Feature Selection
        self.fs = CancelOut(activation='sigmoid', cancelout_loss=True, lambda_1=0.002, lambda_2=0.001)
        self.reshape_lstm = tf.keras.layers.Reshape((window, feature_size))

        self.flatten = tf.keras.layers.Flatten()

        # Part for extract features
        self.tcn = TCN(64, return_sequences=True)
        self.tcn2 = TCN(64, return_sequences=False)
        self.dense = tf.keras.layers.Dense(128, activation='relu')

        # Post Merging
        self.dense_pm = tf.keras.layers.Dense(128, activation='relu')
        self.dense_pm1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense_pm2 = tf.keras.layers.Dense(32, activation='relu')
        self.dense_pm3 = tf.keras.layers.Dense(1, activation='linear')

    def call(self, x):
        '''
          x = (Features TCN sized)
        '''
        # Extract features
        # f = self.flat_lstm(features)
        # f = self.fs(f)
        # f = self.reshape_lstm(f)
        f = self.flatten(x)
        f = self.fs(f)
        f = self.reshape_lstm(f)
        f = self.tcn(f)
        f = self.tcn2(f)
        f = self.dense(f)

        # Post Merging
        f = self.dense_pm(f)
        f = self.dense_pm1(f)
        f = self.dense_pm2(f)
        f = self.dense_pm3(f)

        return f


In [6]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, img, x_data, y_data, batch_size):
        self.img = img
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x_data) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.x_data[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y_data[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_img = np.repeat(np.expand_dims(self.img, axis=0), len(batch_x), axis=0)
        return (batch_img, batch_x), batch_y

# Avec Topographie

In [11]:
taille_fenetre = 18
horizon = 24
ville = 'ajaccio'
image = '110'

In [12]:
FEATURES = ['u100', 'v100', 'u10', 'v10', 't2m', 'i10fg', 'index_hours', 'index_days', 'sp', 'msl', 'd2m']
TARGET = 'A10'

img = iio.imread(f"img/{ville}/{image}.jpg").astype(np.float32)
img = tf.convert_to_tensor(img)

df_test = pd.read_csv(f"data/test_{ville}.csv")

df_test_features = df_test[FEATURES].to_numpy()
df_test_target = df_test[[TARGET]].to_numpy()

def creer_sequences(features, target, taille_fenetre, horizon):
      x, y = [], []
      for i in range(len(features) - taille_fenetre - horizon + 1):
        x.append(features[i:(i + taille_fenetre)])
        y.append(target[i + taille_fenetre + horizon - 1])
      return np.array(x), np.array(y)

x_test, y_test = creer_sequences(df_test_features, df_test_target, taille_fenetre, horizon)
batch_size = 32
test_generator = DataGenerator(img, x_test, y_test, batch_size)

model = MyModelTCN(taille_fenetre, len(FEATURES))
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mape', 'mae', tf.keras.metrics.RootMeanSquaredError()])
#build
model((tf.random.uniform((batch_size, int(image), int(image), 3)) , tf.random.uniform((batch_size, taille_fenetre, len(FEATURES)))))
print("model builded")

/tmp/ipykernel_870/2944221117.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(f"img/{ville}/{image}.jpg").astype(np.float32)


model builded


In [13]:
model.load_weights(f"model_trained/{ville}/checkpoint_tcn_A10_{ville}_amplitude_zoom_{image}_{horizon}h.weights.h5")
model.evaluate(test_generator)

300/300 [==============================] - 3s 8ms/step - loss: 3.9444 - mse: 3.9320 - mape: 81.4355 - mae: 1.3828 - root_mean_squared_error: 1.9829


[3.9443509578704834,
 3.9319519996643066,
 81.43547821044922,
 1.382767677307129,
 1.9829150438308716]

# Sans Topographie

In [13]:
taille_fenetre = 18
horizon = 24
ville = 'bastia'

In [ ]:
FEATURES = ['u100', 'v100', 'u10', 'v10', 't2m', 'i10fg', 'index_hours', 'index_days', 'sp', 'msl', 'd2m']
TARGET = 'A10'

df_test = pd.read_csv(f"data/test_{ville}.csv")

df_test_features = df_test[FEATURES].to_numpy()
df_test_target = df_test[[TARGET]].to_numpy()

def creer_sequences(features, target, taille_fenetre, horizon):
      x, y = [], []
      for i in range(len(features) - taille_fenetre - horizon + 1):
        x.append(features[i:(i + taille_fenetre)])
        y.append(target[i + taille_fenetre + horizon - 1])
      return np.array(x), np.array(y)

x_test, y_test = creer_sequences(df_test_features, df_test_target, taille_fenetre, horizon)
batch_size = 32

model = MyModelTCNNotHybrid(taille_fenetre, len(FEATURES))
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mape', 'mae', tf.keras.metrics.RootMeanSquaredError()])

model(tf.random.uniform((batch_size, taille_fenetre, len(FEATURES))))
print("model builded")

NameError: name 'MyModelTCNNotHybrid' is not defined

In [107]:
model.load_weights(f"model_trained/{ville}/checkpoint_tcn_A10_{ville}_amplitude_without_topo_{horizon}h.weights.h5")
model.evaluate(x_test, y_test)

300/300 [==============================] - 3s 7ms/step - loss: 3.8585 - mse: 3.7806 - mape: 90.5942 - mae: 1.4802 - root_mean_squared_error: 1.9444


[3.858537197113037,
 3.7806177139282227,
 90.59416961669922,
 1.4801870584487915,
 1.9443809986114502]

# Persistance

In [15]:
class Persistance(tf.keras.layers.Layer):
    '''
    Persistance Layer
    '''
    def __init__(self):
        super(Persistance, self).__init__(trainable=False)

    # def build(self, input_shape):
    #     pass
        
    def call(self, inputs):
        return inputs

In [26]:
ville = 'ajaccio'
taille_fenetre = 1

In [27]:
rmse = []
for horizon in [1,3,6,12,24]:
  TARGET = 'A10'

  df_test = pd.read_csv(f"data/test_{ville}.csv")
  df_test_target = df_test[[TARGET]].to_numpy()
  df_test_features = df_test[[TARGET]].to_numpy()

  model = tf.keras.Sequential(
              [
                  Persistance(),
              ]
          )
  model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mape', 'mae', tf.keras.metrics.RootMeanSquaredError()])

  def creer_sequences(features, target, taille_fenetre, horizon):
        x, y = [], []
        for i in range(len(features) - taille_fenetre - horizon + 1):
          x.append(features[i:(i + taille_fenetre)])
          y.append(target[i + taille_fenetre + horizon - 1])
        return np.array(x), np.array(y)

  x_test, y_test = creer_sequences(df_test_features, df_test_target, taille_fenetre, horizon)

  rmse.append(model.evaluate(x_test, y_test)[4])
rmse

301/301 [==============================] - 1s 2ms/step - loss: 6.3539 - mse: 6.3539 - mape: 89.1965 - mae: 1.7137 - root_mean_squared_error: 2.5207


[0.57654869556427,
 1.1399524211883545,
 1.579726219177246,
 2.0415661334991455,
 2.5206854343414307]